# Projet 4 : Anticiper les besoins en consommation électrique de bâtiments
*Pierre-Eloi Ragetly*

Ce projet fait parti du parcours *DataScientist* d'OpenClassrooms.

L'objectif principal est de trouver un modèle permettant de prédire **les émissions de CO2 et la consommation totale d’énergie de bâtiments non destinés à l'habitation.**

Pour cela nous disposons des données de la ville de Seattle pour les années 2015 et 2016. Ces données sont à récupérer sur le site kaggle.

# Partie I : Data Wrangling

Ce notebook a pour but de présenter les données et de décrire les opérations nécessaire au nettoyage afin d'obtenir un format adapté à l'analyse et la modélisation.

In [1]:
# Import des librairies usuelles
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
import pandas as pd
import seaborn as sns

In [2]:
# Change some default parameters of matplotlib using seaborn
plt.rcParams.update(plt.rcParamsDefault)
plt.rcParams.update({'axes.titleweight': 'bold'})
sns.set(style='ticks')
current_palette = sns.color_palette('RdBu')
sns.set_palette(current_palette)

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Récupération-des-données" data-toc-modified-id="Récupération-des-données-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Récupération des données</a></span></li><li><span><a href="#Comparaison-des-deux-jeux-de-données" data-toc-modified-id="Comparaison-des-deux-jeux-de-données-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Comparaison des deux jeux de données</a></span><ul class="toc-item"><li><span><a href="#Variables-n'étant-pas-présentes-dans-l'un-des-jeux-de-données" data-toc-modified-id="Variables-n'étant-pas-présentes-dans-l'un-des-jeux-de-données-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Variables n'étant pas présentes dans l'un des jeux de données</a></span></li><li><span><a href="#Traitement-de-la-variable-localisation" data-toc-modified-id="Traitement-de-la-variable-localisation-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Traitement de la variable localisation</a></span></li><li><span><a href="#Renommer-les-variables-de-2015-comme-celles-de-2016" data-toc-modified-id="Renommer-les-variables-de-2015-comme-celles-de-2016-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Renommer les variables de 2015 comme celles de 2016</a></span></li><li><span><a href="#Variables-non-présentes-dans-le-jeu-de-données-2016" data-toc-modified-id="Variables-non-présentes-dans-le-jeu-de-données-2016-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Variables non présentes dans le jeu de données 2016</a></span></li><li><span><a href="#Ajout-des-variables-manquantes-pour-le-jeu-de-données-de-2016" data-toc-modified-id="Ajout-des-variables-manquantes-pour-le-jeu-de-données-de-2016-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Ajout des variables manquantes pour le jeu de données de 2016</a></span></li><li><span><a href="#Traitement-de-la-variable-OtherFuelUse(kBtu)" data-toc-modified-id="Traitement-de-la-variable-OtherFuelUse(kBtu)-2.6"><span class="toc-item-num">2.6&nbsp;&nbsp;</span>Traitement de la variable OtherFuelUse(kBtu)</a></span></li></ul></li></ul></div>

## Récupération des données

Nous commençons par télécharger les données du site *kaggle*. Ces données sont contenus dans 2 fichier csv :
- 2015-building-energy-benchmarking
- 2016-building-energy-benchmarking

Notre but étant de prédire des données de consommation, avoir des données pour deux dates différentes est un gros plus. **Nous allons pouvoir utiliser les donnée de 2015 (l'année la plus ancienne) comme jeu d'entraînement et les données de 2016 (les plus récentes) comme jeu de test.** Les données étant déja contenues dans deux jeux de données distincts, nous n'aurons pas à séparer les données en deux (test et entraînement).

Une fois téléchargées, nous pouvons les charger dans des DataFrame.

In [3]:
raw_data_2015 = pd.read_csv('data/2015-building-energy-benchmarking.csv')
print(raw_data_2015.shape)
raw_data_2015.head()

(3340, 47)


,OSEBuildingID,DataYear,BuildingType,PrimaryPropertyType,PropertyName,TaxParcelIdentificationNumber,Location,CouncilDistrictCode,Neighborhood,YearBuilt,...,GHGEmissionsIntensity(kgCO2e/ft2),DefaultData,Comment,ComplianceStatus,Outlier,2010 Census Tracts,Seattle Police Department Micro Community Policing Plan Areas,City Council Districts,SPD Beats,Zip Codes
0,1,2015,NonResidential,Hotel,MAYFLOWER PARK HOTEL,659000030,"{'latitude': '47.61219025', 'longitude': '-122...",7,DOWNTOWN,1927,...,2.64,No,NaN,Compliant,NaN,NaN,14.0,NaN,31.0,18081
1,2,2015,NonResidential,Hotel,PARAMOUNT HOTEL,659000220,"{'latitude': '47.61310583', 'longitude': '-122...",7,DOWNTOWN,1996,...,2.38,No,NaN,Compliant,NaN,NaN,14.0,NaN,31.0,18081
2,3,2015,NonResidential,Hotel,WESTIN HOTEL,659000475,"{'latitude': '47.61334897', 'longitude': '-122...",7,DOWNTOWN,1969,...,1.92,Yes,NaN,Compliant,NaN,NaN,56.0,NaN,31.0,18081
3,5,2015,NonResidential,Hotel,HOTEL MAX,659000640,"{'latitude': '47.61421585', 'longitude': '-122...",7,DOWNTOWN,1926,...,31.38,No,NaN,Compliant,High Outlier,NaN,56.0,NaN,31.0,18081
4,8,2015,NonResidential,Hotel,WARWICK SEATTLE HOTEL,659000970,"{'latitude': '47.6137544', 'longitude': '-122....",7,DOWNTOWN,1980,...,4.02,No,NaN,Compliant,NaN,NaN,56.0,NaN,31.0,19576


In [4]:
raw_data_2016 = pd.read_csv('data/2016-building-energy-benchmarking.csv')
print(raw_data_2016.shape)
raw_data_2016.head()

(3376, 46)


,OSEBuildingID,DataYear,BuildingType,PrimaryPropertyType,PropertyName,Address,City,State,ZipCode,TaxParcelIdentificationNumber,...,Electricity(kWh),Electricity(kBtu),NaturalGas(therms),NaturalGas(kBtu),DefaultData,Comments,ComplianceStatus,Outlier,TotalGHGEmissions,GHGEmissionsIntensity
0,1,2016,NonResidential,Hotel,Mayflower park hotel,405 Olive way,Seattle,WA,98101.0,0659000030,...,1.156514e+06,3946027.0,12764.52930,1276453.0,False,NaN,Compliant,NaN,249.98,2.83
1,2,2016,NonResidential,Hotel,Paramount Hotel,724 Pine street,Seattle,WA,98101.0,0659000220,...,9.504252e+05,3242851.0,51450.81641,5145082.0,False,NaN,Compliant,NaN,295.86,2.86
2,3,2016,NonResidential,Hotel,5673-The Westin Seattle,1900 5th Avenue,Seattle,WA,98101.0,0659000475,...,1.451544e+07,49526664.0,14938.00000,1493800.0,False,NaN,Compliant,NaN,2089.28,2.19
3,5,2016,NonResidential,Hotel,HOTEL MAX,620 STEWART ST,Seattle,WA,98101.0,0659000640,...,8.115253e+05,2768924.0,18112.13086,1811213.0,False,NaN,Compliant,NaN,286.43,4.67
4,8,2016,NonResidential,Hotel,WARWICK SEATTLE HOTEL (ID8),401 LENORA ST,Seattle,WA,98121.0,0659000970,...,1.573449e+06,5368607.0,88039.98438,8803998.0,False,NaN,Compliant,NaN,505.01,2.88


## Comparaison des deux jeux de données

### Variables n'étant pas présentes dans l'un des jeux de données

Les deux jeux de données possédent un nombre d'observations similaires, en revanche, nous notons qu'ils ne possédent pas le même nombre de variables. Or, il est indispensable que les deux possédent exactement les mêmes caractéristiques pour pouvoir entraîner (sur les données de 2015) et tester (sur les données de 2016) notre modèle.

On va donc commencer par ne garder que les variables communes aux deux années.

In [5]:
features_2015 = raw_data_2015.columns.tolist()
features_2016 = raw_data_2016.columns.tolist()
features = [c for c in features_2016 if c in features_2015]
print(len(features))

37


Dix-neuf variables ne sont pas présentes dans l'un des deux jeux de données, cela représente plus de 30% de nos variables, c'est loin d'être négligeable. Avant de les écarter, regardons les en détail pour voir si elles ne contiennent pas d'éléments clés.

In [6]:
feat_to_del = [c for c in features_2015 if c not in features] + [c for c in features_2016 if c not in features]
print(len(feat_to_del))
print(feat_to_del)

19
['Location', 'OtherFuelUse(kBtu)', 'GHGEmissions(MetricTonsCO2e)', 'GHGEmissionsIntensity(kgCO2e/ft2)', 'Comment', '2010 Census Tracts', 'Seattle Police Department Micro Community Policing Plan Areas', 'City Council Districts', 'SPD Beats', 'Zip Codes', 'Address', 'City', 'State', 'ZipCode', 'Latitude', 'Longitude', 'Comments', 'TotalGHGEmissions', 'GHGEmissionsIntensity']


Nous remarquons que certaines variables semblent très proches :
- Location & Latitude/Longitude
- GHGEmissions(MetricTonsCO2e) & TotalGHGEmissions
- GHGEmissionsIntensity(kgCO2e/ft2) & GHGEmissionsIntensity
- Zip Codes & ZipCode
- Comment & Comments

Commençons par vérifier si ces variables ne seraient pas les mêmes, mais avec des noms différents.

In [7]:
feat_to_check = ['Location', 'Latitude', 'Longitude',
                 'GHGEmissions(MetricTonsCO2e)', 'TotalGHGEmissions',
                 'GHGEmissionsIntensity(kgCO2e/ft2)', 'GHGEmissionsIntensity',
                 'Zip Codes', 'ZipCode',
                 'Comment', 'Comments']
key = 'OSEBuildingID'
feat_to_check += [key]

# keep only the features to be checked for each dataset
df_2015 = raw_data_2015.loc[:, raw_data_2015.columns.isin(feat_to_check)]
df_2016 = raw_data_2016.loc[:, raw_data_2016.columns.isin(feat_to_check)]

# use the feature 'OSEBuildingID' to merge datasets
df = pd.merge(df_2015, df_2016, on=key)
df.head()

,OSEBuildingID,Location,GHGEmissions(MetricTonsCO2e),GHGEmissionsIntensity(kgCO2e/ft2),Comment,Zip Codes,ZipCode,Latitude,Longitude,Comments,TotalGHGEmissions,GHGEmissionsIntensity
0,1,"{'latitude': '47.61219025', 'longitude': '-122...",249.43,2.64,NaN,18081,98101.0,47.61220,-122.33799,NaN,249.98,2.83
1,2,"{'latitude': '47.61310583', 'longitude': '-122...",263.51,2.38,NaN,18081,98101.0,47.61317,-122.33393,NaN,295.86,2.86
2,3,"{'latitude': '47.61334897', 'longitude': '-122...",2061.48,1.92,NaN,18081,98101.0,47.61393,-122.33810,NaN,2089.28,2.19
3,5,"{'latitude': '47.61421585', 'longitude': '-122...",1936.34,31.38,NaN,18081,98101.0,47.61412,-122.33664,NaN,286.43,4.67
4,8,"{'latitude': '47.6137544', 'longitude': '-122....",507.70,4.02,NaN,19576,98121.0,47.61375,-122.34047,NaN,505.01,2.88


En dehors des variables ZipCodes et ZipCode, il s'agit bien des mêmes variables. En fait la variable **Localisation** de 2015 contient toutes les variables suivantes :
- Latitude
- Longitude
- Adress
- City
- State
- ZipCode

Extrayons toutes ces variables de **Localisation**.

### Traitement de la variable localisation

Comme nous avons précédemment, plusieurs variables de 2016 sont réunis dans la variable *Localisation* sous forme de dictionnaire. Nous allons recréer ces variables en les extrayant de *Localisation*.

In [8]:
import ast

data_2015 = pd.concat([raw_data_2015.drop('Location', axis=1), # Delete 'Location' column
                       raw_data_2015['Location'].apply(ast.literal_eval) # Convert string into dictionary
                                                .apply(pd.Series)], # Split the dictionary into columns
                      axis=1)

Il reste encore à séparer les données de human_address en plusieurs colonnes.

In [9]:
data_2015 = pd.concat([data_2015.drop('human_address', axis=1), # Delete 'human_address' column
                       data_2015['human_address'].apply(ast.literal_eval) # Convert string into dictionary
                                                 .apply(pd.Series)], # Split the dictionary into columns
                      axis=1)

### Renommer les variables de 2015 comme celles de 2016

In [10]:
data_2015.rename(columns={'GHGEmissions(MetricTonsCO2e)': 'TotalGHGEmissions',
                  'GHGEmissionsIntensity(kgCO2e/ft2)': 'GHGEmissionsIntensity',
                  'Comment': 'Comments',
                  'latitude': 'Latitude',
                  'longitude': 'Longitude',
                  'address': 'Address',
                  'city': 'City',
                  'state': 'State',
                  'zip': 'ZipCode'},
                 inplace=True)

Il ne reste plus qu'à vérifier que toutes les variables de 2016 sont bien présentes dans le jeu de données de 2015.

In [11]:
features_2015 = data_2015.columns.tolist()
features_2016 = raw_data_2016.columns.tolist()
features = [c for c in features_2016 if c in features_2015]
print(len(features))

46


C'est bon, toutes les variables de 2016 sont bien présentes dans le jeu de données de 2015. Il nous reste maintenant à traiter les variables de 2015 non présentes dans le jeu de données de 2016.

### Variables non présentes dans le jeu de données 2016

In [12]:
features = [c for c in features_2015 if c not in features_2016]
print(features)

['OtherFuelUse(kBtu)', '2010 Census Tracts', 'Seattle Police Department Micro Community Policing Plan Areas', 'City Council Districts', 'SPD Beats', 'Zip Codes']


Voici la liste des variables manquantes :

Ayant accès à la localisation il est peu pertinent de les garder. C'est d'ailleurs le cas pour toutes les variables ci dessous :
- OtherFuelUse(kBtu)
- 2010 Census Tracts
- Seattle Police Department Micro Community Policing Plan Areas
- City Council Districts
- SPD Beats (ils s'agit de zone de délimitation, elles sont au nombre de cinq à Seattle)
- Zip Codes

Les cinq dernières ne devraient pas changer d'une année à l'autre, et peuvent donc être intégrée au jeu de données de 2016 en reprenant les même valeurs que celles de 2015.

En revanche, ce n'est pas le cas pour **OtherFuelUse(kBtu)**. Il faudra vérifier si on ne peut pas l'obtenir via d'autres variables. Si ce n'est pas le cas, alors il faudra voir si ses valeurs sont négligeables par rapport aux autres sources de consommation.

### Ajout des variables manquantes pour le jeu de données de 2016

In [13]:
feat_to_add =['OSEBuildingID', '2010 Census Tracts',
              'Seattle Police Department Micro Community Policing Plan Areas',
              'City Council Districts',
              'SPD Beats',
              'Zip Codes']

data_2016 = pd.merge(raw_data_2016, data_2015[feat_to_add], how='left', on='OSEBuildingID')

### Traitement de la variable OtherFuelUse(kBtu)

Nous allons commencer par regarder s'il est possible de calculer cette variable à partir de la consommation totale et des autres sources d'énergie.

Puis nous diviserons la différence entre le calcul et la variable *OtherFuelUse(kBtu)* par la consommation totale *SiteEnergyUse(kBtu)*. Cela permettra de vérifier si la différence est négligeable et donc de valider le calcul.

In [14]:
df = data_2015[['SiteEnergyUse(kBtu)', 'SiteEnergyUseWN(kBtu)',
                'SteamUse(kBtu)', 'Electricity(kBtu)', 'NaturalGas(kBtu)', 'OtherFuelUse(kBtu)']].copy()

df['Other_calc'] = df.iloc[:, 0] - df.iloc[:, 2:].sum(axis=1)
df['Other_calc'] = df['Other_calc'] / df.iloc[:, 0]

In [15]:
df.describe()

,SiteEnergyUse(kBtu),SiteEnergyUseWN(kBtu),SteamUse(kBtu),Electricity(kBtu),NaturalGas(kBtu),OtherFuelUse(kBtu),Other_calc
count,3.330000e+03,3.330000e+03,3.330000e+03,3.330000e+03,3.330000e+03,3.330000e+03,3328.000000
mean,4.983106e+06,5.203055e+06,2.502851e+05,3.473209e+06,1.253304e+06,7.142301e+03,-0.000246
std,1.375330e+07,1.400757e+07,3.481900e+06,9.519639e+06,4.415142e+06,1.962790e+05,0.007625
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-0.314228
25%,9.139300e+05,9.883248e+05,0.000000e+00,6.363322e+05,0.000000e+00,0.000000e+00,-0.000041
50%,1.776219e+06,1.953996e+06,0.000000e+00,1.160236e+06,2.945770e+05,0.000000e+00,-0.000033
75%,4.044277e+06,4.368462e+06,0.000000e+00,2.750847e+06,1.115366e+06,0.000000e+00,-0.000022
max,2.958126e+08,2.977410e+08,1.278697e+08,2.847263e+08,1.364484e+08,8.269669e+06,0.000001


Pour la majorité des bâtiments, la différence est marginale par rapport à la consommation totale (<0.1%). A noter que les valeurs obtenues après cacul sont sytématiquement plus faibles .

Nous observons de plus, que pour au moins une des observations, la différence est significative, représentant plus de 30% de la consommation totale.  
Vérifions s'il s'agit d'un cas isolé en affichant toutes les observations dont la différence représente plus de 1% de la consommation totale.

In [16]:
outliers = df[df['Other_calc'].abs()>0.001]
print(outliers.shape[0])
outliers

3


,SiteEnergyUse(kBtu),SiteEnergyUseWN(kBtu),SteamUse(kBtu),Electricity(kBtu),NaturalGas(kBtu),OtherFuelUse(kBtu),Other_calc
564,5288304.0,5288304.0,0.0,5288523.0,618364.0,0.0,-0.116972
1201,1554939.0,2129737.0,0.0,1482658.0,560886.0,0.0,-0.314228
1308,4217108.0,4217108.0,0.0,4217284.0,1201563.0,0.0,-0.284968


Ces valeurs anomaliques sont seulement au nombre de trois. En y regardant de plus près, nous notons que pour deux d'entre elles la consommation en électricité seule est supérieure à la consommation totale, ce qui est impossible. Il s'agit probablement d'outliers, vérifions s'ils ont été étiquetés comme tels grâce à la variable *Outlier*.

In [17]:
idx = outliers.index
data_2015.loc[idx, ['SiteEnergyUse(kBtu)', 'Electricity(kBtu)', 'Outlier']]

,SiteEnergyUse(kBtu),Electricity(kBtu),Outlier
564,5288304.0,5288523.0,NaN
1201,1554939.0,1482658.0,NaN
1308,4217108.0,4217284.0,NaN


Ces observations n'ont pas été étiquetées comme outliers, mais elles le sont p

In [18]:
test_elec = data_2015['Electricity(kBtu)'] / data_2015['Electricity(kWh)']
test_elec.describe()

count    3328.000000
mean        3.412217
std         0.004412
min         3.411612
25%         3.412139
50%         3.412142
75%         3.412143
max         3.666667
dtype: float64

In [27]:
test_gas = data_2015['NaturalGas(kBtu)'] / data_2015['NaturalGas(therms)']
test_gas.describe()

count    2074.000000
mean      100.027864
std         0.838721
min        91.250000
25%        99.997525
50%       100.000000
75%       100.002271
max       122.000000
dtype: float64

In [28]:
test_tot = data_2015['SiteEnergyUse(kBtu)'] - data_2015['SiteEUI(kBtu/sf)']*data_2015['PropertyGFABuilding(s)']
test_tot = test_tot/data_2015['SiteEnergyUse(kBtu)']
test_tot.describe()

count    3328.000000
mean       -0.021069
std         0.249590
min        -4.581702
25%        -0.002220
50%        -0.000047
75%         0.001617
max         2.295078
dtype: float64

In [29]:
test_tot[test_tot.abs()>0.01].size

1429

In [23]:
test_gfa = data_2015['PropertyGFATotal'] - (data_2015['PropertyGFAParking']+data_2015['PropertyGFABuilding(s)'])
test_gfa = test_tot/data_2015['PropertyGFATotal']
test_gfa.describe()

count    3328.000000
mean       -0.000042
std         0.000032
min        -0.000343
25%        -0.000062
50%        -0.000034
75%        -0.000017
max         0.000027
dtype: float64

In [24]:
data_2015['Outlier'].value_counts()

High Outlier    46
Low Outlier     38
Name: Outlier, dtype: int64

In [25]:
data_2015.columns

Index(['OSEBuildingID', 'DataYear', 'BuildingType', 'PrimaryPropertyType',
       'PropertyName', 'TaxParcelIdentificationNumber', 'CouncilDistrictCode',
       'Neighborhood', 'YearBuilt', 'NumberofBuildings', 'NumberofFloors',
       'PropertyGFATotal', 'PropertyGFAParking', 'PropertyGFABuilding(s)',
       'ListOfAllPropertyUseTypes', 'LargestPropertyUseType',
       'LargestPropertyUseTypeGFA', 'SecondLargestPropertyUseType',
       'SecondLargestPropertyUseTypeGFA', 'ThirdLargestPropertyUseType',
       'ThirdLargestPropertyUseTypeGFA', 'YearsENERGYSTARCertified',
       'ENERGYSTARScore', 'SiteEUI(kBtu/sf)', 'SiteEUIWN(kBtu/sf)',
       'SourceEUI(kBtu/sf)', 'SourceEUIWN(kBtu/sf)', 'SiteEnergyUse(kBtu)',
       'SiteEnergyUseWN(kBtu)', 'SteamUse(kBtu)', 'Electricity(kWh)',
       'Electricity(kBtu)', 'NaturalGas(therms)', 'NaturalGas(kBtu)',
       'OtherFuelUse(kBtu)', 'TotalGHGEmissions', 'GHGEmissionsIntensity',
       'DefaultData', 'Comments', 'ComplianceStatus', 'Outlier',
